In [ ]:
import ovation.contents as contents

from ovation.session import connect

from pprint import pprint
from getpass import getpass

## Connect

In [ ]:
email = input('Ovation email: ')

In [ ]:
session = connect(email)

## Walk directory

Starting with a parent directory or project...

In [ ]:
parent_id = input('Parent UUID: ')

In [ ]:
parent = session.get(session.entity_path('folders', parent_id))

Walk the contents of `parent` recursively and sum the total byte size of each file's most recent ("head") revision. If the metadata is not fully updated on the Revision, call the `upload-complete` handler to update its metadat from S3.

In [ ]:
total_bytes = 0
for (parent, folders, files) in contents.walk(session, parent):
    for f in files:
        rev = contents.get_head_revision(session, f)
        if 'content_length' not in rev.attributes:
            session.put(rev.links['upload-complete']) # This updates Revision from S3 metadata
            rev = contents.get_head_revision(session, f)

        pprint("{}: {} bytes".format(rev.attributes.name, rev.attributes.content_length))
        total_bytes += rev.attributes.get('content_length', 0)

print("Total bytes: {}".format(total_bytes))